In [55]:
from pprint import pprint as pp
#       1  2  3  4
bffs = [2, 3, 4, 1]
bffs = [3, 3, 4, 1]
bffs = [2, 1, 1, 2]
bffs = [2, 1, 1, 2, 6, 5, 5, 6]
# bffs = [3, 3, 4, 3]

class TreeNode(object):
    bffBy = None
    id = None
    myBff = None
    
    def __init__(self, bffBy=None, id=None, myBff=None):
        if bffBy is None:
            bffBy = []
        self.bffBy = bffBy
        self.id = id
        self.myBff = myBff
        self._depth = None
        
    def __str__(self):
        return '<Kid id="{id}", bffby={bffBy}, myBff={myBff}>'.format(
            id=self.id, 
            bffBy=[bff.id for bff in self.bffBy] if self.bffBy else self.bffBy,
            myBff=self.myBff.id if self.myBff else self.myBff)
    
    def __repr__(self):
        return str(self)
    
    @property
    def depth(self):
        # print 'depth ' + str(self.id)
        if self._depth is None:
            if self.bffBy is None or len(self.bffBy) == 0:
                self._depth = 1
            else:
                self._depth = max(bff.depth for bff in self.bffBy) + 1
        return self._depth
    
    def isGluable(self):
        return False

    
    
class SuperNode(TreeNode):
    def __init__(self, **kwargs):
        self._loops = kwargs['loops']
        kwargs.pop('loops')
        super(SuperNode, self).__init__(**kwargs)
        
    def __str__(self):
        return '<SuperNode id="{id}", isGluable={isGluable}, bffby={bffBy}, myBff={myBff}, loops={loops}>'.format(
            id=self.id, 
            isGluable=self.isGluable(),
            bffBy=[bff.id for bff in self.bffBy] if self.bffBy else self.bffBy,
            myBff=self.myBff.id if self.myBff else self.myBff,
            loops=self._loops
        )

    @property
    def depth(self):
        # print 'SuperNode depth'
        if len(self._loops) > 2:
            return 1
        print self.bffBy[0]
        return (self.bffBy[0].depth+1, self.bffBy[1].depth+1)

    @property
    def length(self):
        # print 'SuperNode length'
        if len(self._loops) > 2:
            return len(self._loops)
        kid1Depth = max([bffBy.depth for bffBy in self._loops[0].bffBy if bffBy not in self._loops] or [0])
        kid2Depth = max([bffBy.depth for bffBy in self._loops[1].bffBy if bffBy not in self._loops] or [0])
        return kid1Depth + 2 + kid2Depth
    
    def isGluable(self):
        return len(self._loops) == 2
    
    
    
def buildTreesFromList(bffs):
    trees = {}
    nodes = {}
    for i, myBff in enumerate(bffs):
        me = i + 1
        if me not in nodes:
            nodes[me] = TreeNode(id=me)
            
        if myBff not in nodes:
            nodes[myBff] = TreeNode(id=myBff)
        
        nodes[me].myBff = nodes[myBff]
        nodes[myBff].bffBy.append(nodes[me])
        
    visited = set()
    for kid in nodes.values():
        if kid.id in visited:
            continue
        # let traverse the kid chain
        print 'current kid'
        print kid.id
        visitedKids = set()
        pathIds = []
        while True:
            visited.add(kid.id)
            visitedKids.add(kid.id)
            pathIds.append(kid.id)
            nextKid = kid.myBff
            if nextKid is None:
                break
            if nextKid.id in visitedKids:
                # looped
                start = pathIds.index(nextKid.id)
                kidIdsInLoop = pathIds[start:]
                print 'kidIdsInLoop' 
                print kidIdsInLoop
                treeId = ','.join(map(str, sorted(kidIdsInLoop)))
                if treeId in trees:
                    break
                # Find the loop create a root node
                kidsInLoop = [nodes[id] for id in kidIdsInLoop]
                rootNode = SuperNode(loops=kidsInLoop, id=treeId)
                trees[treeId] = rootNode
                # replace all child nodes' bff to this super node
                for k in kidsInLoop:
                    print 'replacing bff'
                    for bffBy in k.bffBy:
                        if bffBy not in kidsInLoop:
                            # print 'numa numa'
                            bffBy.myBff = rootNode
                            # print bffBy
                            rootNode.bffBy.append(bffBy)
                break
            kid = nextKid
    print 'nodes'
    pp(nodes)
    #loners = {id: kid for id, kid in nodes.iteritems() if kid.bffBy is None or len(kid.bffBy) == 0}
    #pp(loners)
    
    return trees
    
    
trees = buildTreesFromList(bffs)
print 'trees'
pp(trees)
# for tree in trees.values():
#     print 'depth'
#     print tree.depth
#     print 'isGluable'
#     print tree.isGluable()
#     print 'length'
#     print tree.length
    
    
gluableLength = 0
nonGluableLength = 0
for t in trees.values():
    if t.isGluable():
        gluableLength += t.length
    else:
        nonGluableLength = max(nonGluableLength, t.length)
print gluableLength
print nonGluableLength
print max(gluableLength, nonGluableLength)

current kid
1
kidIdsInLoop
[1, 2]
replacing bff
replacing bff
current kid
3
current kid
4
current kid
5
kidIdsInLoop
[5, 6]
replacing bff
replacing bff
current kid
7
current kid
8
nodes
{1: <Kid id="1", bffby=[2, 3], myBff=2>,
 2: <Kid id="2", bffby=[1, 4], myBff=1>,
 3: <Kid id="3", bffby=[], myBff=1,2>,
 4: <Kid id="4", bffby=[], myBff=1,2>,
 5: <Kid id="5", bffby=[6, 7], myBff=6>,
 6: <Kid id="6", bffby=[5, 8], myBff=5>,
 7: <Kid id="7", bffby=[], myBff=5,6>,
 8: <Kid id="8", bffby=[], myBff=5,6>}
trees
{'1,2': <SuperNode id="1,2", isGluable=True, bffby=[3, 4], myBff=None, loops=[<Kid id="1", bffby=[2, 3], myBff=2>, <Kid id="2", bffby=[1, 4], myBff=1>]>,
 '5,6': <SuperNode id="5,6", isGluable=True, bffby=[7, 8], myBff=None, loops=[<Kid id="5", bffby=[6, 7], myBff=6>, <Kid id="6", bffby=[5, 8], myBff=5>]>}
8
0
8
